# NBA Player Touches Analysis Project (Buy Low/Sell High Detector)

## Project Overview
This project uses web scraping and data analysis to track changes in NBA players' game involvement, specifically focusing on the number of touches per player. The aim is to identify players whose roles in their teams have evolved, making them potentially valuable assets in fantasy leagues as buy-low candidates.

## Project Goals
- **Analyze Player Role Changes**: By observing shifts in the number of touches from one season to the next, we can detect players who are becoming more involved in the game (i.e., making more plays) or, conversely, who may be seeing reduced roles.
- **Fantasy Basketball Insights**: Use data insights to inform fantasy basketball players on potential buy-low candidates.

## Main Features
1. **Data Scraping**:
   - Automate the extraction of player data for two specified NBA seasons.
   - Retrieve season statistics, including touches, team, and other relevant metrics.

2. **Data Processing**:
   - Convert scraped data into a structured format using pandas DataFrames.
   - Merge datasets for different seasons to perform comparative analysis.
   
3. **Difference Calculation**:
   - Calculate changes in player touches across seasons.
   - Filter results to show significant increases or decreases, helping fantasy players identify role changes.

## How to Use
1. Set the target seasons in the `get_nba_stats_table()` function and run the code cell.
2. The resulting table displays players with noticeable changes in touches, helping you identify potential buy-low targets.

## Future Improvements
- **Historical Analysis**: Add more seasons to track player involvement trends over multiple years.
- **Additional Metrics**: Incorporate other stats (e.g., points per touch, assists) to provide a fuller picture of player performance.
- **Visualization**: Create graphs to illustrate player role changes over time.

## Dependencies
- `selenium`: For automated web scraping.
- `pandas`: For data manipulation and analysis.

---
**Note**: This project currently scrapes data from nba.com, which may require you to install a WebDriver compatible with your version of Chrome.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

def get_nba_stats_table(season):
    """
    Retrieves NBA player statistics from the NBA website for a given season.
    
    This function opens the NBA stats page for the specified season, dismisses any
    cookie policy banners if present, scrolls to the "All" option to load all data rows,
    and parses the player statistics table into a pandas DataFrame.
    
    Parameters:
        season (str): The NBA season to retrieve data for, in the format "YYYY-YY" (e.g., "2022-23").
        
    Returns:
        pd.DataFrame: A pandas DataFrame containing the player statistics table, 
                      with an additional 'SEASON' column for the specified season.
    """
    
    # Initialize the Chrome WebDriver and set a wait time for elements to load.
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 15)
    
    # Go to the NBA stats page for the specified season.
    url = f"https://www.nba.com/stats/players/touches?Season={season}"
    driver.get(url)
    
    # Close the cookie consent banner if it appears.
    try:
        time.sleep(3)  # Wait briefly to ensure the page has loaded
        close_button = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.ID, "onetrust-close-btn-container"))
        )
        close_button.click()
        print("Cookie policy banner closed.")
    except:
        # If the banner is not present or fails to close, proceed without error.
        print("Cookie banner not found or already closed.")

    time.sleep(3)  # Wait briefly to ensure changes take effect after closing the banner
        
    # Scroll to and select "All" option to display all rows in the table.
    try:
        # Locate the dropdown menu for changing the pagination.
        dropdown = wait.until(
            EC.presence_of_element_located((By.XPATH, "//div[@class='Pagination_pageDropdown__KgjBU']//select[@class='DropDown_select__4pIg9']"))
        )
        
        # Scroll the dropdown into view to ensure it's visible on the screen.
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", dropdown)
        
        # Click on the dropdown to expand the options.
        dropdown.click()
        print("Clicked dropdown for 'All'.")

        # Select the "All" option to load all rows in the table.
        option_all = wait.until(EC.element_to_be_clickable((By.XPATH, "//option[@value='-1']")))
        option_all.click()
        print("'All' option selected.")

        # Wait for the table to load all rows by checking for a minimum number of rows.
        wait.until(lambda driver: len(driver.find_elements(By.CSS_SELECTOR, ".Crom_table__p1iZz tr")) > 100)
        print("All rows loaded.")
    except Exception as e:
        print("Error selecting 'All' option:", e)
        
    # Parse the table data into headers and rows.
    headers, table_data = [], []
    try:
        # Wait for the main stats table to be present on the page.
        table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'Crom_table__p1iZz')))
        
        # Get all rows from the table.
        rows = table.find_elements(By.TAG_NAME, "tr")
            
        # Extract header text from the first row (header row).
        headers = [cell.text.replace("\n", " ").strip() for cell in rows[0].find_elements(By.TAG_NAME, "th")]

        # Extract each cell's text data for each row (excluding the header).
        table_data = [
            [cell.text for cell in row.find_elements(By.TAG_NAME, "td")]
            for row in rows[1:] if row.find_elements(By.TAG_NAME, "td")  # Ensure rows are not empty
        ]
    except Exception as e:
        print("Error parsing table:", e)
        
    # Create a DataFrame from the table data and add a 'SEASON' column.
    df = pd.DataFrame(table_data, columns=headers)
    df.insert(1, 'SEASON', season)  # Insert season column after the player name column
    
    # Close the WebDriver session.
    driver.quit()
    print("Driver closed.")
    
    return df

In [2]:
# Retrieve data for the 2023-24 and 2024-25 seasons
df_2023_24 = get_nba_stats_table("2023-24")
df_2024_25 = get_nba_stats_table("2024-25")

# Convert the TOUCHES column to a numeric type for both DataFrames
df_2023_24["TOUCHES"] = pd.to_numeric(df_2023_24["TOUCHES"], errors="coerce")
df_2024_25["TOUCHES"] = pd.to_numeric(df_2024_25["TOUCHES"], errors="coerce")

# Merge data by players to align data from both seasons
df_merged = pd.merge(
    df_2024_25[['PLAYER', 'MIN', 'TOUCHES']],  # Data from the 2024-25 season
    df_2023_24[['PLAYER', 'MIN', 'TOUCHES']],  # Data from the 2023-24 season
    on="PLAYER",                        # Join based on PLAYER column
    suffixes=('_2024_25', '_2023_24')   # Add suffixes to differentiate TOUCHES columns by season
)

# Calculate the change in touches
df_merged["TOUCHES_CHANGE"] = df_merged["TOUCHES_2024_25"] - df_merged["TOUCHES_2023_24"]

# Optionally, save the result to a CSV file
# df_merged.to_csv("touches_comparison.csv", index=False)

Cookie policy banner closed.
Clicked dropdown for 'All'.
'All' option selected.
All rows loaded.
Driver closed.
Cookie policy banner closed.
Clicked dropdown for 'All'.
'All' option selected.
All rows loaded.
Driver closed.


## Key Data Insights
To identify players whose roles have evolved significantly, the following two tables are generated:

1. **Top 25 Players by Touches in the 2024-25 Season**  
   Using `df_merged[df_merged['TOUCHES_2024_25'] > 20].sort_values('TOUCHES_2024_25', ascending=False).head(25)`, this table displays players with the highest touches for the 2024-25 season. By focusing on players with more than 20 touches, this list highlights those actively involved in their teams' play.

2. **Top 25 Players by Touches Change Between Seasons**  
   Using `df_merged[df_merged['TOUCHES_2024_25'] > 20].sort_values('TOUCHES_CHANGE', ascending=False).head(25)`, this table displays players with the most significant increase in touches compared to the previous season. A large positive change suggests an increase in involvement, which can be valuable for identifying potential buy-low candidates in fantasy basketball.

## Interpretation and Next Steps
These tables offer an initial look at players with changing roles. A high number of touches or a significant increase in touches could indicate a player taking on more offensive responsibility. However, users should conduct further analysis—such as evaluating minutes played, team strategy, and other performance metrics—to make informed decisions.

## Disclaimer
This project is intended as a tool for data-driven insights rather than definitive fantasy basketball advice. The analysis highlights trends that may warrant further investigation but is not a substitute for comprehensive player evaluation. I am not responsible for specific decisions based on this data, as it is ultimately a preliminary analysis meant to provide a foundation for more in-depth research.


In [3]:
df_merged[df_merged['TOUCHES_2024_25'] > 20].sort_values('TOUCHES_2024_25', ascending = False).head(25)

,PLAYER,MIN_2024_25,TOUCHES_2024_25,MIN_2023_24,TOUCHES_2023_24,TOUCHES_CHANGE
292,Nikola Jokić,38.1,115.8,34.7,101.1,14.7
163,Jalen Johnson,36.7,99.3,33.7,69.7,29.6
358,Trae Young,36.6,97.5,36.0,85.5,12.0
369,Tyrese Haliburton,34.8,94.1,32.2,90.9,3.2
46,Cade Cunningham,36.2,93.8,33.5,80.1,13.7
333,Scottie Barnes,33.6,90.0,34.9,75.1,14.9
370,Tyrese Maxey,39.7,89.9,37.6,87.4,2.5
238,Keyonte George,31.9,89.2,26.8,63.0,26.2
247,LaMelo Ball,33.6,89.1,32.3,83.8,5.3
78,Damian Lillard,35.8,88.3,35.3,78.9,9.4


In [4]:
df_merged[df_merged['TOUCHES_2024_25'] > 20].sort_values('TOUCHES_CHANGE', ascending = False).head(25)

,PLAYER,MIN_2024_25,TOUCHES_2024_25,MIN_2023_24,TOUCHES_2023_24,TOUCHES_CHANGE
36,Brandon Boston Jr.,26.8,53.8,10.8,16.5,37.3
86,Davion Mitchell,28.4,64.8,15.3,28.4,36.4
163,Jalen Johnson,36.7,99.3,33.7,69.7,29.6
238,Keyonte George,31.9,89.2,26.8,63.0,26.2
205,Jordan Hawkins,30.3,49.0,17.4,22.9,26.1
317,RJ Barrett,33.0,78.7,31.7,53.0,25.7
166,Jalen Suggs,29.1,71.4,27.0,45.8,25.6
145,Ivica Zubac,35.0,66.3,26.4,42.8,23.5
19,Anthony Black,24.6,49.1,16.9,26.4,22.7
132,Gradey Dick,32.4,43.9,20.8,21.5,22.4
